In [1]:
#load all necessaries files 
import time
init_time = time.time()

import numpy as np

path = "/home/micael/MyWorkspace/DATA"

signal = np.load(path+"/"+"lvbb125Matrix.npy")[0:5000,:]

ZZ = np.load(path+"/"+"ZZMatrix.npy")[0:5000,:]

TTbar = np.load(path+"/"+"TTbarMatrix.npy")[0:5000,:]

bkg = np.append(ZZ,TTbar,axis=0)

all_data = np.append(signal,bkg,axis=0)

current_time = time.time()
print 'Time to load all files: %1.3f seconds'%(current_time-init_time)

Time to load all files: 0.072 seconds


In [18]:
from sklearn.cluster import KMeans
from numpy import linalg as LA
import matplotlib.pyplot as plt


from sklearn.datasets.samples_generator import make_blobs

plt.figure()

X, y = make_blobs(n_samples=2, centers=5, n_features=5,
                  random_state=0)

B = np.array([[1, 2],[3, 4]])

# baricentro 

def Baricentro(A):
    M = np.sum(A,axis=0)/A.shape[0]
    print A
    print 'Baricentro do conjunto: ', M
    return M

b = Baricentro(B)
print b

def DispTotal(data,dataCenter):
    dispTotal = 0
    for i in range(data.shape[0]):
        dispTotal += LA.norm(data[i,:]-dataCenter)
    print 'DisperTotal do conjunto: ', dispTotal
    return dispTotal

DispTotal(B,Baricentro(B))

[[1 2]
 [3 4]]
Baricentro do conjunto:  [2 3]
[2 3]
[[1 2]
 [3 4]]
Baricentro do conjunto:  [2 3]
DisperTotal do conjunto:  2.82842712475


2.8284271247461903

In [1]:
import numpy as np
from functions import ClusterMerits, AuxiliarFunctions
from numpy import linalg as LA


In [3]:
B = np.array([[1, 2],[3, 4]])
#dataCenter = ClusterMerits.Baricentro(B)

def ClassDiameter(data,dataCenter):
    phi = []
    for i in range(B.shape[0]):
        phi.append(LA.norm(B[i,:]-dataCenter))
    print 'Class Diameter: ', max(phi)
    return max(phi)

ClusterMerits.DispTotal(B,ClusterMerits.Baricenter(B),media='')
ClusterMerits.DispTotal(B,ClusterMerits.Baricenter(B),media='m')


Baricenter of set:  [2 3]
Total Dispersion of set:  4.0
Baricenter of set:  [2 3]
Mean Total Dispersion of set:  2.0


2.0000000000000004

In [2]:
B = np.array([[4, 3],[7, 2]])

ClusterMerits.ClassDiameter(B,ClusterMerits.Baricenter(B))


Baricenter of set:  [5 2]
Class Diameter:  2.0


2.0